In [1]:
from urllib import request
import json
from io import StringIO
import os

import requests
from bs4 import BeautifulSoup

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import demoji

import sys
import random

%matplotlib inline

from IPython.display import JSON

demoji.download_codes()
access_token = os.environ['ACCESS_TOKEN']

... OK (Got response in 0.48 seconds)
Writing emoji data to C:\Users\prayt\.demoji/codes.json ...
... OK


# Save or load raw data

In [2]:
# raw_data = {'raw_data':ads_df}
# with open('raw_data.pickle', 'wb') as f:
#     pickle.dump(raw_data, f, pickle.HIGHEST_PROTOCOL)

# experiment_pickle = {'experiment_data':experimental_search}
# with open('experiment_pickle.pickle', 'wb') as f:
#     pickle.dump(experiment_pickle, f, pickle.HIGHEST_PROTOCOL)

# joined_congress_candidates = {'joined_congress_candidates':experiment_2}
# with open('joined_congress_candidates.pickle', 'wb') as f:
#     pickle.dump(joined_congress_candidates, f, pickle.HIGHEST_PROTOCOL)

## emoji removed labeled leans for test/train data merged to df appended, urls removed, 
# emoji_removed = {'emoji_removed':experiment_2,
#                 'entity_labels':entity_leans}
# with open('emoji_removed.pickle', 'wb') as f:
#     pickle.dump(emoji_removed, f, pickle.HIGHEST_PROTOCOL)

## special_char_removed - this is post emoji removal
# special_char_removed = {'special_char_removed':experiment_2,
#                 'entity_labels':entity_leans}
# with open('special_char_removed.pickle', 'wb') as f:
#     pickle.dump(special_char_removed, f, pickle.HIGHEST_PROTOCOL)

# with open('special_char_removed.pickle', 'rb') as f:
#     a_pickle = pickle.load(f)
# experiment_2 = a_pickle['special_char_removed']
# # congress = a_pickle['congress']


In [14]:
# This function is helpful when labeling entity leans

def lookup_funder(funder):
    return experiment_2.loc[experiment_2['funding_entity']==funder]['ad_creative_body']
    
#lookup_funder("The Family Protection Association")#[56383]

In [15]:
def scrape_ads(srch_term, num_scrapes=8, search_term_lean=None):
    
    url='https://graph.facebook.com/v4.0/ads_archive/'

    response = requests.get(url,params={"access_token":access_token,
                        "ad_type":"POLITICAL_AND_ISSUE_ADS",
                        "search_terms":srch_term,
                        "ad_reached_countries":"['US']",
                        "fields":"ad_creative_link_description,ad_creative_link_title,funding_entity,ad_creative_body,ad_snapshot_url",
                        "limit":200
                        })
        
    if len(response.json())!=2:
        print('200 results or less')
        return
    
    
    ad_data = pd.DataFrame(response.json()['data'])
    
    for x in list(range(0,num_scrapes)):
        response = requests.get(response.json()['paging']['next'])
        #print(x)
        #print(response.json().keys())
        if len(response.json())!=2:
            print('bingo', x)
            ad_data['srch_term'] = srch_term
            return ad_data
        else:
            ad_data = ad_data.append(response.json()['data'], ignore_index=True)
    
    ad_data['srch_term'] = srch_term
    
#     if search_term_lean != None:
#         ad_data['srch_term_lean'] = srch_term_lean
    
    return ad_data


def scrape_topics(topic_list, num_scrapes=8):
    #calls scrape ads for multiple search terms
    #topic list must be list of strings
    ads_df=pd.DataFrame()
    for topic in topic_list:
        try:
            a = scrape_ads(topic, num_scrapes)
            ads_df = ads_df.append(a, ignore_index=True)
        except: 
            print('error in topic: ' + topic)
    return ads_df


In [16]:
# Remove URL Function
def remove_url(text):
    # findall() has been used  
    # with valid conditions for urls in string 
    text = re.sub('HTTP[S]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))\S+','', str(text))
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))\S+','', str(text))
    text = re.sub('www.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))\S+','', str(text))
    text = re.sub('WWW.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))\S+','', str(text))
    text = re.sub('bit.ly\S+','', text)  
    return text 

def remove_links(df, column):
    #column must be string
    df[column] = df[column].map(lambda x: remove_url(x))
    return df

In [17]:
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    text = str(text)
    # Convert words to lower case and split them
    text = text.lower().split()

#     # Optionally, remove stop words
#     if remove_stopwords:
#         stops = set(stopwords.words("english"))
#         text = [w for w in text if not w in stops]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    #text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

#     # Optionally, shorten words to their stems
#     if stem_words:
#         text = text.split()
#         stemmer = SnowballStemmer('english')
#         stemmed_words = [stemmer.stem(word) for word in text]
#         text = " ".join(stemmed_words)
    
    return(text)

def clean_all_docs(df, column):
    print('got this far')
    df[column] = df[column].map(lambda x: text_to_wordlist(x))
    return df

In [18]:
# Remove emojis and replace with descriptor for one string

def replace_an_emoji(text):
    #depencies - install and import demoji
    temp_dict = demoji.findall(text)
    for emoji in temp_dict.keys():
        text = text.replace(emoji, temp_dict[emoji])
    return text

# Remove emojis and replace with descriptor for dataframe column
def replace_all_emoji(df, column):
    #column must be string
    df[column] = df[column].map(lambda x: replace_an_emoji(x))
    return df

## Call API for all keywords below

In [22]:
srch_topics = ['socialist','trade war', 'wall', 'medicare', 
                'invasion','veterans', 'mexico', 'me too', 'gay rights', 
                'black lives matter', 'white nationalism', 'terrorism', 'corrupt',
                'muslim brotherhood', 'global warming', 'climate change', 'trade', 'trade war',
                'china', 'farmers','religious liberty', 'nato','russia', 'elections','hillary',
                'benghazi', 'space force', 'pro-life', 'abortion','pro-choice', 'RBG', 'kavanaugh',
                'feel the bern', 'middle east', 'military spending', 'president',
                'obama', 'mitch mcconnell', 'AOC', 'squad', 'court','blue lives matter',
                'schumer', 'kamala harris', 'biden', 'GOP', 'democratic','conservative',
                'liberal', 'DNC', 'arpaio', 'gun violence', '2nd ammendment','maga',
                'trump', 'kaepernick','feminism','science','antifa','tea party', 'dc statehood',
                'dc statehood','diversity', 'vulnerable','sexual assault', 'safe space', 'elites',
                '99%', 'safety net', 'welfare', 'trickle down', 'student loans', 'social security',
                'obamacare', 'affordable care act', 'opioid crisis','kim jong un', 'putin', 'flag',
                'tlaib', 'ilhan omar','israel','lincoln monument', '4th of july', 'baltimore', 'charlottesville',
                'pelosi','hannity', 'maddow','warren','native american','pocahantas','standing rock','pollution',
                'devin nunes', 'john mccain', 'epa', 'cfpb', 'wall street','elijah cummings','conspiracy',
                'hate speech', 'amazon', 'big tech', 'zuckerberg','campaign finance', 'citizens united', 'maxine waters',
                'enemy of the people', 'journalism','democratic debate','debt ceiling', 'sherrod brown', 'ted cruz',
                'beto','iran','obstruction of justice','endangered species','environment','nuclear weapons',
                'brexit','racism','bias','coal', 'west', 'sovereign citizen', 'cliven bundy']

scraped_ads =  scrape_topics(srch_topics, num_scrapes=200)

## Prelim data cleaning to remove duplicates - each NLP modeling technique requires different data cleaning steps so more cleaning will occur later

In [ ]:
scraped_ads = remove_links(scraped_ads.ad_creative_body)
scraped_ads = scraped_ads.drop_duplicates(subset=['ad_creative_body'])

## Save scraped ads in pickle for use later

In [ ]:
scraped_ads = {'scraped_ads':scraped_ads}
with open('scraped_ads.pickle', 'wb') as f:
    pickle.dump(scraped_ads, f, pickle.HIGHEST_PROTOCOL)

## Sort funding entities by the number of unique ads in the dataset and save to CSV for identification of left wing and right wing funding entities.  From here left/right wing entities need to be labeled by hand and loaded back in the section under 'Create Training data labels

In [97]:
csv_df = pd.DataFrame(scraped_ads.funding_entity.value_counts()).reset_index()
csv_df.to_csv(r'~/projects_folder/side_projects_B/ad_recommender/label_entities_file.csv')

# Create Training data labels

## Create left wing/right wing ad labels by merging labeled funding entities csv

In [ ]:
with open('scraped_ads.pickle', 'rb') as f:
    a_pickle = pickle.load(f)
scraped_ads = a_pickle['scraped_ads']
entity_leans = pd.read_csv('label_entities_file.csv', encoding = "ISO-8859-1");
scraped_ads = scraped_ads.merge(entity_leans, how='left', on='funding_entity')

## Subset and clean labeled train/test data for LSTM NN, export to csv, aws_model_file_x picks it up from here.  

In [ ]:
labeled_data = scraped_ads.loc[(scraped_ads['lean']=='d')|(scraped_ads['lean']=='r'), ['ad_creative_body','lean']].reset_index().drop(columns='index')
labeled_data = remove_links(labeled_data, 'ad_creative_body')
labeled_data = labeled_data.drop_duplicates(subset='ad_creative_body')
labeled_data = replace_all_emoji(labeled_data, 'ad_creative_body')
labeled_data = clean_all_docs(labeled_data, 'ad_creative_body')
labeled_data.loc[labeled_data['lean']=='d','lean'] = 1
labeled_data.loc[labeled_data['lean']=='r','lean'] = 0
labeled_data.to_csv('C:/Users/prayt/projects_folder/side_projects_B/ad_recommender/labeled_data.csv')